### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [ ]:
from modelModule.model2 import VAE2
import torch

In [ ]:
model = VAE2()
a = torch.randn((10,57))
b = torch.randn((10,57))
model(a,b)

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Letter_train.csv')
print(dataset.Max_Val)
print(dataset.Min_Val)

In [ ]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
batch.keys()

### 模型精度测定

In [1]:
from dataModule.dataset1 import FlatDataset
from modelModule.model1 import VAE
# from modelModule.model2 import VAE2
from torch.utils.data import DataLoader
from utils import restore_data, result_show, get_missing
import torch
import numpy as np

In [2]:
## 模型参数载入
model = VAE(dim=16, nhead=4)
checkpoint_path = r'模型参数保存\Letter_lr_0_0001_lr_scheduler_CosineAnnealingLR\version_2\checkpoints\epoch=10-step=693.ckpt'    
checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(r'C:\王源权\西南财经大学\硕士毕业论文\模型代码\VAE_pytorchLightning\Letter_train.csv', data_norm='mean_norm')
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

载入成功


In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

#### 另一种验证方式

In [5]:
test_dataset = FlatDataset('Letter_test.csv', data_norm='minmax_norm') # minmax_norm
src_data = np.stack([test_dataset[index]['src_data'] for index in range(10)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(10)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan
imputed_data, output = model.inference(input, miss_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))

In [7]:
output

tensor([[0.2658, 0.4721, 0.3511, 0.3595, 0.2420, 0.4375, 0.4803, 0.2847, 0.3454,
         0.5506, 0.4162, 0.5300, 0.1948, 0.5613, 0.2584, 0.5404],
        [0.2699, 0.4680, 0.3490, 0.3588, 0.2405, 0.4363, 0.4807, 0.2866, 0.3437,
         0.5478, 0.4170, 0.5286, 0.1941, 0.5614, 0.2568, 0.5398],
        [0.2713, 0.4698, 0.3494, 0.3583, 0.2425, 0.4361, 0.4808, 0.2858, 0.3464,
         0.5453, 0.4163, 0.5328, 0.1933, 0.5624, 0.2573, 0.5395],
        [0.2696, 0.4703, 0.3512, 0.3578, 0.2405, 0.4353, 0.4794, 0.2868, 0.3458,
         0.5495, 0.4175, 0.5320, 0.1937, 0.5614, 0.2562, 0.5405],
        [0.2704, 0.4638, 0.3483, 0.3575, 0.2391, 0.4337, 0.4803, 0.2863, 0.3436,
         0.5491, 0.4165, 0.5304, 0.1948, 0.5617, 0.2567, 0.5375],
        [0.2696, 0.4689, 0.3507, 0.3580, 0.2422, 0.4359, 0.4783, 0.2864, 0.3433,
         0.5490, 0.4175, 0.5283, 0.1952, 0.5622, 0.2567, 0.5382],
        [0.2680, 0.4695, 0.3508, 0.3598, 0.2416, 0.4374, 0.4794, 0.2846, 0.3447,
         0.5509, 0.4165, 0.5319, 0.19

In [6]:
normal_data

array([[0.2142857 , 0.19999999, 0.28571427, 0.13333332, 0.06666666,
        0.26666665, 0.73333328, 0.13333332, 0.46666664, 0.73333328,
        0.66666662, 0.28571427, 0.06666666, 0.71428566, 0.14285713,
        0.28571427],
       [0.2142857 , 0.33333331, 0.4285714 , 0.26666665, 0.26666665,
        0.59999996, 0.39999997, 0.19999999, 0.39999997, 0.66666662,
        0.33333331, 0.4285714 , 0.13333332, 0.49999996, 0.35714283,
        0.57142853],
       [0.28571427, 0.5333333 , 0.28571427, 0.33333331, 0.13333332,
        0.19999999, 0.5333333 , 0.5333333 , 0.13333332, 0.46666664,
        0.33333331, 0.71428566, 0.26666665, 0.49999996, 0.2142857 ,
        0.6428571 ],
       [0.35714283, 0.66666662, 0.49999996, 0.46666664, 0.26666665,
        0.26666665, 0.66666662, 0.13333332, 0.5333333 , 0.73333328,
        0.79999995, 0.57142853, 0.19999999, 0.57142853, 0.14285713,
        0.35714283],
       [0.28571427, 0.46666664, 0.28571427, 0.5333333 , 0.33333331,
        0.5333333 , 0.46666664, 

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))